In [1]:
def jaccard_similarity(set1, set2):
    set1_upper = {element.upper() for element in set1}
    set2_upper = {element.upper() for element in set2}

    intersection = len(set1_upper.intersection(set2_upper))
    union = len(set1_upper.union(set2_upper))
    
    return intersection / union if union != 0 else 0

In [3]:
sim = jaccard_similarity('dog', 'dogo')

In [ ]:
def find_trigrams(text: str) -> set:
    """
    returns a set of ngrams for the given string
    :param text: the string to find ngrams for
    :param number: the length the ngrams should be. defaults to 3 (trigrams)
    :return: set of ngram strings
    """

    if not text:
        return set()

    words = [f'  {x} ' for x in re.split(r'\W+', text.lower()) if x.strip()]

    ngrams = set()

    for word in words:
        for x in range(0, len(word) - 4):
            ngrams.add(word[x:x+3])

    return ngrams